In [7]:
import pandas as pd

df=pd.read_csv("merged_importance_zeroshot.csv")

#drop source human
df = df[df['source'] != 'human']


#save to csv
df.to_csv("importance_zeroshot.csv", index=False)

df.shape

(13200, 11)

In [8]:
#concatnate two datasets
import pandas as pd

zero  = "importance_zeroshot.csv"
context  = "importance_context.csv"
OUT_FILE = "merged_importance_zero_context.csv"

# load
zero = pd.read_csv(zero)
context = pd.read_csv(context)

# keep exactly these shared columns (in this order)
cols = [
    'row_id','source','base_model','variant_id','model',
    'dc_solution','rating','label','condition','iteration','timestamp'
]

# some columns may be missing in human (base_model/variant_id/model/iteration/timestamp) → create if needed
for c in cols:
    if c not in zero.columns: zero[c] = pd.NA
    if c not in context.columns: context[c] = pd.NA

merged = pd.concat([context[cols], zero[cols]], ignore_index=True)

merged.to_csv(OUT_FILE, index=False)
print("Saved:", OUT_FILE, "rows:", len(merged))

Saved: merged_importance_zero_context.csv rows: 26400


In [ ]:
import pandas as pd
df=pd.read_csv("merged_importance_zero_context.csv")

#rename value in column 'base_model' from 'gemma3-12b' to 'gemma3', and 'llama-pro' to 'llama'
df['base_model'] = df['base_model'].replace({'gemma': 'gemma3', 'llama-pro': 'llama'})


#SAVE TO CSV
df.to_csv("merged_importance_zero_context.csv", index=False)

In [ ]:
import pandas as pd

# df = pd.read_csv("your_file.csv")
pattern = r'^(?P<head>[^_\n]*?)-[^_\n]+(?=_)'
repl    = r'\g<head>'

df["variant_id"] = df["variant_id"].astype(str).str.replace(pattern, repl, regex=True)
df["row_id"]     = df["row_id"].astype(str).str.replace(pattern, repl, regex=True)

df.to_csv("new_merged_importance_zero_context.csv", index=False)
#this new_merged...csv, came into use due to challenge of naming that happened in two base models: gemma3, and llama
#therefore this is the correct version to use for the analysis

In [3]:
import pandas as pd
df = pd.read_csv("new_merged_importance_zero_context.csv")

#select distinct values in column 'base_model'
df['base_model'].unique()
df['condition'].unique()

print(df['base_model'].unique())
print(df['condition'].unique())


['phi4' 'llama' 'mistral' 'gemma3']
['CONTEXT' 'ZEROSHOT']


In [ ]:
#comparing distributions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("new_merged_importance_zero_context.csv")

needed = {"base_model","variant_id","dc_solution","rating","condition","iteration","timestamp"}
missing = needed - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

df["condition"] = df["condition"].str.upper().str.replace(" ", "")
df = df[df["condition"].isin(["ZEROSHOT","CONTEXT"])].copy()

df["rating"] = pd.to_numeric(df["rating"], errors="coerce").astype("Int64")
df = df[df["rating"].between(0,4)].copy()

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
# Keep the last occurrence if duplicates for the same key
df = (df.sort_values("timestamp")
        .groupby(["base_model","variant_id","dc_solution","condition","iteration"], as_index=False)
        .tail(1))

# ===================== PAIRED BY ITERATION =====================
pairs = (df.pivot_table(
            index=["base_model","variant_id","dc_solution","iteration"],
            columns="condition",
            values="rating",
            aggfunc="first")
         .reset_index())

pairs = pairs.dropna(subset=["ZEROSHOT","CONTEXT"]).copy()
pairs["delta"] = pairs["CONTEXT"] - pairs["ZEROSHOT"]

# Diagnostics (expect 50 pairs per base_model • variant • dc_solution)
expected_per_solution = 50
coverage = (pairs.groupby(["base_model","variant_id","dc_solution"])["delta"]
                 .size().rename("n_pairs").reset_index())
coverage["coverage_ok"] = coverage["n_pairs"].eq(expected_per_solution)
print("\n=== Coverage sample ===")
print(coverage.head())

# ===================== SUMMARY HELPERS (MEDIANS + proportions) =====================
def summarize_median(paired_df, keys):
    g = paired_df.copy()
    # proportions of sign changes (paired)
    change = np.select([g["delta"]>0, g["delta"]<0], ["improved","not improved"], default="same")
    g["change"] = change
    props = (g.groupby(keys + ["change"]).size()
               .unstack(fill_value=0))
    props = props.div(props.sum(axis=1), axis=0)
    props = props.rename(columns=lambda c: f"prop_{c}")

    # medians (per condition) and median delta
    med = (g.groupby(keys)
             .agg(median_zeroshot=("ZEROSHOT","median"),
                  median_context=("CONTEXT","median"),
                  median_delta=("delta","median"),
                  n_pairs=("delta","size"))
             .reset_index())
    res = med.merge(props.reset_index(), on=keys, how="left")
    for col in ["prop_improved","prop_same","prop_not_improved"]:
        if col not in res:
            res[col] = 0.0
    return res

# ---------- (A) BY BASE MODEL ----------
by_bm = summarize_median(pairs, ["base_model"]).sort_values("base_model")
print("\n=== Paired summary (Medians) — BY BASE MODEL ===")
print(by_bm[["base_model","n_pairs","median_zeroshot","median_context","median_delta",
             "prop_improved","prop_same","prop_not_improved"]])

# ---------- (B) BY DC SOLUTION (across all base models & variants) ----------
by_solution = summarize_median(pairs, ["dc_solution"]).sort_values("dc_solution")
print("\n=== Paired summary (Medians) — BY DC SOLUTION ===")
print(by_solution[["dc_solution","n_pairs","median_zeroshot","median_context","median_delta",
                   "prop_improved","prop_same","prop_not_improved"]])

# (Optional) If you’d also like base_model × dc_solution:
# by_bm_sol = summarize_median(pairs, ["base_model","dc_solution"])

# Stacked proportions 0–4 per condition
def stacked_props(data, group_cols, title):
    rating_levels = [0,1,2,3,4]
    ct = (data.groupby(group_cols + ["rating"])
              .size()
              .unstack(fill_value=0)
              .reindex(columns=rating_levels, fill_value=0))
    prop = ct.div(ct.sum(axis=1), axis=0)
    ax = prop.plot(kind="bar", stacked=True, figsize=(9,4))
    ax.set_title(title)
    ax.set_xlabel(" • ".join(group_cols))
    ax.set_ylabel("Proportion")
    ax.legend(title="Rating", bbox_to_anchor=(1.02,1), loc="upper left")
    plt.tight_layout(); plt.show()

# Per BASE MODEL
for bm, g in df.groupby("base_model"):
    stacked_props(g, ["condition"], f"Rating distribution (0–4) — {bm} (all variants, solutions, iterations)")

# Per DC SOLUTION
for sol, g in df.groupby("dc_solution"):
    stacked_props(g, ["condition"], f"Rating distribution (0–4) — {sol} (all base models, variants, iterations)")

# ECDF unpaired view
def plot_ecdf(series, label):
    x = np.sort(series.astype(float))
    y = np.arange(1, len(x)+1) / len(x)
    plt.step(x, y, where="post", label=label)

# ECDF per BASE MODEL
for bm, g in df.groupby("base_model"):
    plt.figure(figsize=(6,4))
    for cond, gg in g.groupby("condition"):
        plot_ecdf(gg["rating"].values, cond)
    plt.title(f"ECDF — {bm}")
    plt.xlabel("Rating (0–4)"); plt.ylabel("Proportion ≤ rating")
    plt.legend(title="Condition"); plt.tight_layout(); plt.show()

# ECDF per DC SOLUTION
for sol, g in df.groupby("dc_solution"):
    plt.figure(figsize=(6,4))
    for cond, gg in g.groupby("condition"):
        plot_ecdf(gg["rating"].values, cond)
    plt.title(f"ECDF — {sol}")
    plt.xlabel("Rating (0–4)"); plt.ylabel("Proportion ≤ rating")
    plt.legend(title="Condition"); plt.tight_layout(); plt.show()

# Unpaired comparison (CONTEXT vs ZEROSHOT) by base model and by dc solution.
def cliffs_delta_unpaired(x, y):
    x = np.asarray(x); y = np.asarray(y)
    gt = sum((xi > y).sum() for xi in x)
    lt = sum((xi < y).sum() for xi in x)
    n = len(x)*len(y)
    return (gt - lt) / n if n else np.nan

def cliff_by(df, keys):
    rows = []
    for key_vals, g in df.groupby(keys):
        a = g[g["condition"]=="ZEROSHOT"]["rating"].values
        b = g[g["condition"]=="CONTEXT"]["rating"].values
        rows.append({**({k:v for k,v in zip(keys, key_vals)} if isinstance(key_vals, tuple) else {keys[0]: key_vals}),
                     "cliffs_delta": cliffs_delta_unpaired(b, a)})  # positive => CONTEXT higher
    return pd.DataFrame(rows)

cliff_bm  = cliff_by(df, ["base_model"]).sort_values("base_model")
cliff_sol = cliff_by(df, ["dc_solution"]).sort_values("dc_solution")

# Paired summaries (medians)
by_bm.to_csv("summary_by_base_model_medians.csv", index=False)
by_solution.to_csv("summary_by_dc_solution_medians.csv", index=False)

# Coverage of pairs (should be ~50 per variant×solution)
coverage.to_csv("pair_coverage.csv", index=False)

# Optional: full paired rows with deltas
pairs.to_csv("paired_rows_with_delta.csv", index=False)

# Optional nonparametric effect sizes
cliff_bm.to_csv("cliffs_delta_by_base_model.csv", index=False)
cliff_sol.to_csv("cliffs_delta_by_dc_solution.csv", index=False)

print("\n=== Cliff's delta (CONTEXT vs ZEROSHOT) — BY BASE MODEL ===")
print(cliff_bm)
print("\n=== Cliff's delta (CONTEXT vs ZEROSHOT) — BY DC SOLUTION ===")
print(cliff_sol)

**EXPLANATIONS**
1. ECDFs [Eucledian Cliff Delta Functions]: provides a clear view of the ratings and easy to spot ceiling ratings. 
    - if plotted (context vs. zeroshot) and the 'context curve' appearing lower means, more high ratings.
    - easy to indetify whether the differences in ratings are global or not. 
2. Stacked proportions: 
    - proportions of composition of ratings per base model [4's, 3's etc], also which is much more than other
3. Paired deltas (context - zeroshot)
    - pair (same variant x solution x iteration)
    - plots two conditions as side-by-side bars proportions and one after the other
    - median delta: presents the shift (differences between medians of the two conditions)
    - proportions improved/same/worse: between the two conditions
    - pair_id = base_model x variant_id x dc_solution x iteration [two rows per pair, one from each condition]
    - 